In [1]:
%%html
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [2]:
from bokeh.models.widgets import Panel, Tabs
from bokeh.io import output_notebook, output_file, show
from bokeh.plotting import figure, show

from bokeh.models import HoverTool, WheelZoomTool, ResetTool, PanTool,  Label

output_file("tabs.html")
output_notebook()

p1 = figure(plot_width=300, plot_height=300)
p1.circle([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], size=20, color="navy", alpha=0.5)
tab1 = Panel(child=p1, title="circle")

p2 = figure(plot_width=300, plot_height=300)
p2.line([1, 2, 3, 4, 5], [6, 7, 2, 4, 5], line_width=3, color="navy", alpha=0.5)
tab2 = Panel(child=p2, title="line")

#tabs = Tabs(tabs=[ tab1, tab2 ])



# show(tabs)

Loading BokehJS ...

In [3]:
import pandas as pd
import numpy as np

In [67]:
import warnings
warnings.filterwarnings('ignore')

In [68]:
# pull in new data, extreme heat as a tester file
county_hex = '~/Code/jupyter-notebooks/cci-dv-2017/data/county-hex.csv'
cci =  pd.read_csv(county_hex)

# change county column name
cci.columns = ['county' if x=='County' else x for x in cci.columns]

# set index to counties
cci.set_index('county')

cci = cci.sort_values(by = "col-c", ascending=True)

cci = cci[['county','Population','row-c', 'col-c']]

# check out first lines
# cci.head()


In [69]:
# locations of Climate Change Indicator Data
extreme_heat = '~/Code/jupyter-notebooks/cci-dv-2017/data/extreme-heat.csv'
pm25 = '~/Code/jupyter-notebooks/cci-dv-2017/data/pm25.csv'
sea_level_rise ='~/Code/jupyter-notebooks/cci-dv-2017/data/sea-level-rise.csv'
ozone = '~/Code/jupyter-notebooks/cci-dv-2017/data/ozone.csv'
wildfire = '~/Code/jupyter-notebooks/cci-dv-2017/data/wildfire.csv'

red_values_dict = {0: '#fed976', 1: '#feb24c', 2: '#fd8d3c', 3: '#f03b20', 4: '#bd0026'}
purp_values_dict = {0: '#dadaeb', 1: '#bcbddc', 2: '#9e9ac8', 3: '#756bb1', 4: '#54278f'}


In [70]:
# read in extreme heat data set
eh = pd.read_csv(extreme_heat)
eh = eh[eh.ReportYear == 2050]
eh = eh.sort_values(by = "estimate", ascending=True)
#eh = eh.set_index('Geoname')
eh = eh[['County_Name','estimate']]
eh['eh_r'] = eh['estimate'].rank(ascending=1)
eh.columns = ['county' if x=='County_Name' else x for x in eh.columns]
eh.columns = ['eh_i' if x=='estimate' else x for x in eh.columns]
eh = eh.set_index('county')

# creating quintiles
eh['eh_q'] = pd.qcut(eh['eh_i'], 5, labels=False)
eh['eh_color'] = [red_values_dict.get(v, None) for v in eh['eh_q']]

eh['eh_i_round'] = round(eh['eh_i'])

#eh.head()

In [71]:
# Extreme Heat Bar

hover_eh = HoverTool(tooltips=[
    ("County: ", "@county"),
    ("Extreme Heat Days: ", "@eh_i_round")],
     mode='mouse'
                    )

plot_options = dict(width=800, plot_height=300, tools=[WheelZoomTool(), PanTool(),ResetTool(),hover_eh])

e = figure(title='Climate Change Indicator: Estimated Days of Extreme Heat in California 2050, by County', **plot_options)

eh_i_mean = round(np.mean(eh['eh_i'], axis=0),1)

e.xaxis.axis_label = 'County'
e.yaxis.axis_label = '# Extreme Heat Days'

#e.Label(x=70, y=70, text='Some Stuff')

e.vbar(x=eh['eh_r'], width=0.5, bottom=0,
       top=eh['eh_i'], color=eh['eh_color'],source=eh)

e.line(x=[0,58], y=[eh_i_mean,eh_i_mean],line_width=3, line_color='black', line_alpha=0.5)

e.xaxis.visible = True
e.xgrid.visible = False
e.ygrid.visible = False

e.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
e.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
e.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

cci_link = 'Indicator: Projected number of extreme heat days above the 98th percentile for 2050 from computed maximum temperature for each location using 1961-1990 baseline data. More info at: \n https://www.cdph.ca.gov/Programs/OHE/Pages/CC-Health-Vulnerability-Indicators.aspx'

url_e = Label(x=20, y=-50,x_units='screen', y_units='screen',
                 text=cci_link, render_mode='css', text_font_size='12px')

eh_mean = 'Average # of hot days: {} days'.format(eh_i_mean)
e_mean = Label(x=40, y=170,x_units='screen', y_units='screen',
                 text=eh_mean, render_mode='css', text_font_size='12px')

e.add_layout(e_mean)
e.add_layout(url_e)

show(e)

In [12]:
wf =  pd.read_csv(wildfire,encoding ='latin1',low_memory=False)

# rename column names
wf.columns = ['county' if x=='County_Name' else x for x in wf.columns]
wf.columns = ['wf_i' if x=='estimate' else x for x in wf.columns]

wf['wf_i_round'] = round(wf['wf_i'])

# set index to counties
wf.set_index('county')

# grabbing only county geotype and all race
wf = wf[(wf.geotype == 'CO') & (wf.race_eth_name == 'Total')] 

#sort
wf = wf.sort_values(by = "wf_i", ascending=True)
wf['wf_r'] = wf['wf_i'].rank(ascending=1)

# quants
wf['wf_q'] = pd.qcut(wf['wf_i'], 5, labels=False)
wf['wf_color'] = [purp_values_dict.get(v, None) for v in wf['wf_q']]

# just keep three bits
wf = wf[['county', 'wf_r', 'wf_i','wf_color','wf_i_round']]
#wf.head(10)

In [58]:
# Wildfire Bar
hover_wf = HoverTool(tooltips=[
    ("County: ", "@county"),
    ("Wildfire: ", "@wf_i_round")],
     mode='mouse'
                    )

w = figure(title='Climate Change Indicator: % of Population Living Near Area of High Wildfire Risk in California, by County',plot_width=800, plot_height=300, tools=[WheelZoomTool(), PanTool(),ResetTool(),hover_wf])

wf_i_mean = round(np.mean(wf['wf_i'], axis=0),1)

w.vbar(x=range(len(wf['county'])), width=0.5, bottom=0,
       top='wf_i', color=wf['wf_color'], source=wf)

w.line(x=[0,58], y=[wf_i_mean,wf_i_mean],line_width=3, line_color='black', line_alpha=0.5)

w.xaxis.visible = True
w.xgrid.visible = False
w.ygrid.visible = False

w.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
w.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
w.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

w.xaxis.axis_label = 'County'
w.yaxis.axis_label = '% population living w high wildfire risk'

wf_mean = 'Average % of pop at risk: {}% of county'.format(wf_i_mean)
w_mean = Label(x=40, y=50,x_units='screen', y_units='screen',
                 text=wf_mean, render_mode='css', text_font_size='12px')

cci_link = '% population living in very high fire hazard severity zone, more here: https://www.cdph.ca.gov/Programs/OHE/Pages/CC-Health-Vulnerability-Indicators.aspx'

url_w = Label(x=20, y=-50,x_units='screen', y_units='screen',
                 text=cci_link, render_mode='css', text_font_size='12px')

w.add_layout(w_mean)
w.add_layout(url_w)

show(w)

/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [62]:
#reading in and cleaning up ozone data
oz =  pd.read_csv(ozone,encoding ='latin1',low_memory=False)

# rename column names
oz.columns = ['county' if x=='County_Name' else x for x in oz.columns]
oz.columns = ['oz_i' if x=='estimate' else x for x in oz.columns]

oz['oz_i_round'] = round(oz['oz_i'],3)

# set index to counties
oz.set_index('county')

# grabbing only county geotype and all race
oz = oz[(oz.geotype == 'CO') & (oz.race_eth_name == 'Total')] 

# just keep three bits
oz = oz[['county', 'oz_i','oz_i_round']]

#sort
oz = oz.sort_values(by = "oz_i", ascending=True)
oz['oz_r'] = oz['oz_i'].rank(ascending=1)

# quants
oz['oz_q'] = pd.qcut(oz['oz_i'], 5, labels=False)
oz['oz_color'] = [purp_values_dict.get(v, None) for v in oz['oz_q']]

oz.head()


,county,oz_i,oz_i_round,oz_r,oz_q,oz_color
3680,Lake,0.0,0.0,5.5,0.0,#dadaeb
3950,Siskiyou,0.0,0.0,5.5,0.0,#dadaeb
3734,Mendocino,0.0,0.0,5.5,0.0,#dadaeb
3635,Humboldt,0.0,0.0,5.5,0.0,#dadaeb
3923,Santa Cruz,0.0,0.0,5.5,0.0,#dadaeb


In [64]:
#making the ozone bar chart

# Wildfire Bar
hover_oz = HoverTool(tooltips=[
    ("County: ", "@county"),
    ("Ozone: ", "@oz_i_round")],
     mode='mouse'
                    )

o = figure(title='Climate Change Indicator: Ozone in California Counties',plot_width=800, plot_height=300, tools=[WheelZoomTool(), PanTool(),ResetTool(),hover_oz])

oz_i_mean = round(np.mean(oz['oz_i'], axis=0),3)

o.vbar(x=range(len(oz['county'])), width=0.5, bottom=0,
       top='oz_i', color=oz['oz_color'], source=oz)

o.line(x=[0,58], y=[oz_i_mean,oz_i_mean],line_width=3, line_color='black', line_alpha=0.5)

o.xaxis.visible = True
o.xgrid.visible = False
o.ygrid.visible = False

o.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
o.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
o.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

o.xaxis.axis_label = 'County'
o.yaxis.axis_label = 'Three-year ozone concentration exceedance (ppm)'

oz_mean = 'Average Ozone Exceedance: {} ppm'.format(oz_i_mean)
o_mean = Label(x=40, y=50,x_units='screen', y_units='screen',
                 text=oz_mean, render_mode='css', text_font_size='12px')

cci_link = 'Indicator = Amount of the daily maximum 8-hour ozone concentration that exceeds the 8-hour California standard (0.070 ppm), averaged over three years. More info at: (2009 to 2011) https://www.cdph.ca.gov/Programs/OHE/Pages/CC-Health-Vulnerability-Indicators.aspx'

url_o = Label(x=20, y=-50,x_units='screen', y_units='screen',
                 text=cci_link, render_mode='css', text_font_size='12px')

o.add_layout(o_mean)
o.add_layout(url_o)


show(o)

/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
/Users/danielmadrigal/anaconda/lib/python3.6/site-packages/bokeh/util/deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


In [72]:
output_file("tabby.html")

tab1 = Panel(child=w, title="wildfires")

tab2 = Panel(child=e, title="extreme heat")

tab3 = Panel(child=o, title="ozone")


tabs = Tabs(tabs=[ tab1, tab2, tab3])

show(tabs)